This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

**Note** You should select on of the kernels with GDAL installed, eg. "Geo science"

In [1]:
try:
    import eomaji
except ModuleNotFoundError:
    !pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Prepare data for PyDMS and ET Flows

In [2]:
import os 
import openeo
from eomaji.utils import draw_utils, date_selector
from eomaji.utils.general_utils import dump_area_date_info
from eomaji.workflows import prepare_data_cubes

## 1. Select Area of Interest
Draw a polygon on the map, for the area you want to process

In [3]:
map, bboxs = draw_utils.draw_aoi()
map

Map(center=[40, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## 2. Select Date from Available Dates for Sentinel-3
Search for days where sentinel 3 images are available for the given AOI. 

**Note**: max_cloud_cover refers to the full tile’s coverage, not just your AOI, so results may vary.

In [4]:
# Define search parameters
start_date = "2023-05-01"
end_date = "2023-07-30"
bbox = bboxs[-1] # If you didn't draw a polygon, please insert a bbox here in the form of [minx, miny, maxx, maxy]
max_cloud_cover = 10  # Filter out high-cloud-coverage scenes

# Search for available Sentinel-3 imagery
date_selection = date_selector.get_available_dates(
    start_date=start_date,
    end_date=end_date,
    bbox=bbox,
    max_cloud_cover=max_cloud_cover
)

Dropdown(description='Pick a Date:', options=(('2023-05-12', datetime.date(2023, 5, 12)), ('2023-05-31', datet…

## 3. Connect to OpenEO Backend

In [5]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

2025-06-20 07:10:49,906 [INFO] Loaded openEO client config from sources: []
2025-06-20 07:10:50,701 [INFO] Found OIDC providers: ['CDSE']
2025-06-20 07:10:50,704 [INFO] No OIDC provider given, but only one available: 'CDSE'. Using that one.
2025-06-20 07:10:51,046 [INFO] Using default client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e' from OIDC provider 'CDSE' info.
2025-06-20 07:10:51,053 [INFO] Found refresh token: trying refresh token based authentication.
2025-06-20 07:10:51,056 [INFO] Doing 'refresh_token' token request 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'refresh_token'] (client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')
2025-06-20 07:10:51,546 [INFO] Obtained tokens: ['access_token', 'id_token', 'refresh_token']
2025-06-20 07:10:51,549 [INFO] Storing refresh token for issuer 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE' (client 'sh-b1c3a958-52d4-40fe-a333-153

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## 4. Download Sentinel 2 and Sentinel 3 data for the specified AOI and date

In [6]:
data_dir = "./data"
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

In [ ]:
s2_path, s3_path, worldcover_path, dem_s2_path, dem_s3_path, acq_time = prepare_data_cubes.prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date_selection.value,
    sentinel2_search_range = 3,
    out_dir = data_dir,
)

0:00:00 Job 'j-2506200711064e72b1018fe9bd7228fb': send 'start'
0:00:20 Job 'j-2506200711064e72b1018fe9bd7228fb': created (progress 0%)
0:00:25 Job 'j-2506200711064e72b1018fe9bd7228fb': created (progress 0%)
0:00:32 Job 'j-2506200711064e72b1018fe9bd7228fb': created (progress 0%)
0:00:39 Job 'j-2506200711064e72b1018fe9bd7228fb': created (progress 0%)
0:00:49 Job 'j-2506200711064e72b1018fe9bd7228fb': created (progress 0%)
0:01:02 Job 'j-2506200711064e72b1018fe9bd7228fb': queued (progress 0%)
0:01:17 Job 'j-2506200711064e72b1018fe9bd7228fb': queued (progress 0%)
0:01:36 Job 'j-2506200711064e72b1018fe9bd7228fb': queued (progress 0%)
0:02:01 Job 'j-2506200711064e72b1018fe9bd7228fb': queued (progress 0%)
0:02:31 Job 'j-2506200711064e72b1018fe9bd7228fb': running (progress N/A)
0:03:08 Job 'j-2506200711064e72b1018fe9bd7228fb': running (progress N/A)
0:03:55 Job 'j-2506200711064e72b1018fe9bd7228fb': running (progress N/A)
0:04:54 Job 'j-2506200711064e72b1018fe9bd7228fb': finished (progress 100%)

2025-06-20 07:16:00,551 [INFO] Downloading Job result asset 'openEO.nc' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2506200711064e72b1018fe9bd7228fb/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/585a2f339afbefa1fec641916ab39ae9/openEO.nc?expires=1751008560 to data/20230512_e3f1b38e/s2_data.nc
/usr/local/lib/python3.10/dist-packages/openeo/rest/connection.py:1171: UserWarning: Property filtering with unsupported properties according to collection/STAC metadata: {'orbitDirection', 'timeliness'} (supported: dict_keys(['bands', 'eo:bands', 'raster:bands'])).
  return DataCube.load_collection(


0:00:00 Job 'j-2506200716024759ababc9a9a56067ad': send 'start'
0:00:13 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:00:18 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:00:24 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:00:32 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:00:42 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:00:54 Job 'j-2506200716024759ababc9a9a56067ad': queued (progress 0%)
0:01:10 Job 'j-2506200716024759ababc9a9a56067ad': finished (progress 100%)


2025-06-20 07:17:12,906 [INFO] Downloading Job result asset 'openEO.nc' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2506200716024759ababc9a9a56067ad/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/1580cd46ff8c92d711ba3c95ccdb6abd/openEO.nc?expires=1751008632 to data/20230512_e3f1b38e/s3_data.nc


0:00:00 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': send 'start'
0:00:14 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': created (progress 0%)
0:00:19 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': created (progress 0%)
0:00:25 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': created (progress 0%)
0:00:33 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': created (progress 0%)
0:00:43 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': created (progress 0%)
0:00:56 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': running (progress N/A)
0:01:11 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': running (progress N/A)
0:01:30 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': running (progress N/A)
0:01:54 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': running (progress N/A)
0:02:24 Job 'j-2506200717134a2daf4bbcdc7ea35aa1': finished (progress 100%)


2025-06-20 07:19:39,220 [INFO] Downloading Job result asset 'openEO_2011-05-12Z.tif' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2506200717134a2daf4bbcdc7ea35aa1/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/07aa0abd48f00cc93dfa0262f29a86eb/openEO_2011-05-12Z.tif?expires=1751008779 to data/20230512_e3f1b38e/20230512_ELEV.tif


0:00:00 Job 'j-250620071940464cb50a92b8d934ca3a': send 'start'
0:00:13 Job 'j-250620071940464cb50a92b8d934ca3a': created (progress 0%)
0:00:18 Job 'j-250620071940464cb50a92b8d934ca3a': created (progress 0%)
0:00:24 Job 'j-250620071940464cb50a92b8d934ca3a': created (progress 0%)
0:00:32 Job 'j-250620071940464cb50a92b8d934ca3a': running (progress N/A)
0:00:42 Job 'j-250620071940464cb50a92b8d934ca3a': running (progress N/A)
0:00:55 Job 'j-250620071940464cb50a92b8d934ca3a': running (progress N/A)
0:01:10 Job 'j-250620071940464cb50a92b8d934ca3a': running (progress N/A)
0:01:29 Job 'j-250620071940464cb50a92b8d934ca3a': running (progress N/A)
0:01:53 Job 'j-250620071940464cb50a92b8d934ca3a': finished (progress 100%)


2025-06-20 07:21:34,540 [INFO] Downloading Job result asset 'openEO_2011-05-12Z.tif' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-250620071940464cb50a92b8d934ca3a/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/3d7f8e78686b690bd80b356409c0eb48/openEO_2011-05-12Z.tif?expires=1751008894 to data/20230512_e3f1b38e/meteo_dem.tif


0:00:00 Job 'j-250620072135420e843c811e440a6416': send 'start'
0:00:12 Job 'j-250620072135420e843c811e440a6416': created (progress 0%)
0:00:18 Job 'j-250620072135420e843c811e440a6416': created (progress 0%)
0:00:24 Job 'j-250620072135420e843c811e440a6416': created (progress 0%)
0:00:32 Job 'j-250620072135420e843c811e440a6416': created (progress 0%)
0:00:42 Job 'j-250620072135420e843c811e440a6416': running (progress N/A)
0:00:54 Job 'j-250620072135420e843c811e440a6416': running (progress N/A)


### Store AOI and Date for use in other notebooks    

In [ ]:
dump_area_date_info(
    date = date_selection.value, 
    bbox = bbox, 
    out_dir = data_dir
)